# Extraindo informações do site Arbety para pegar o número da sorte 

In [2]:
import time
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Edge
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from bs4 import BeautifulSoup

# Configura as opções do Edge
options = Options()
options.add_argument("--headless")  # Executa o Edge em modo headless

# Configura o serviço do EdgeDriver
service = Service(EdgeChromiumDriverManager().install())

# Cria uma nova instância do Microsoft Edge
driver = Edge(service=service, options=options)

driver.implicitly_wait(10)  # espera até 10 segundos antes de lançar uma NoSuchElementException

url = "https://www.arbety.com/games/double"
driver.get(url)

# Armazena o código HTML atual da página
html_antigo = driver.page_source

try:
    while True:
        try:
            # Verifica se os elementos estão presentes na página
            elemento_pai = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'items')]"))
            )

            # Executa o código para pegar os elementos filhos
            elementos_filhos = elemento_pai.find_elements(By.XPATH, "./*")
            for elemento in elementos_filhos:
                soup = BeautifulSoup(elemento.get_attribute('innerHTML'), 'html.parser')
                div = soup.find('div')
                if div:
                    cor = div.get('class')[1] if len(div.get('class')) > 1 else None
                    aria_label = div.get('aria-label')
                    numero = div.text
                    if aria_label:
                        data, hora = aria_label.split(", ")
                    else:
                        data, hora = None, None
                    print(f"Cor: {cor}, Data: {data}, Hora: {hora}, Número: {numero}")

            # Armazena o código HTML atual da página
            html_atual = driver.page_source

            # Verifica se houve mudanças na página
            if html_atual != html_antigo:
                html_antigo = html_atual
                driver.refresh()

            # Espera por 15 segundos antes de verificar novamente
            time.sleep(15)

        except Exception as e:
            print(e)
            driver.quit()
            break

except KeyboardInterrupt:
    print("Programa interrompido pelo usuário")
    driver.quit()


Cor: green, Data: 25/08/2023, Hora: 09:44:46, Número: 8
Cor: red, Data: 25/08/2023, Hora: 09:45:13, Número: 4
Cor: red, Data: 25/08/2023, Hora: 09:45:39, Número: 5
Cor: green, Data: 25/08/2023, Hora: 09:46:06, Número: 10
Cor: green, Data: 25/08/2023, Hora: 09:46:32, Número: 13
Cor: green, Data: 25/08/2023, Hora: 09:46:59, Número: 11
Cor: green, Data: 25/08/2023, Hora: 09:47:25, Número: 13
Cor: green, Data: 25/08/2023, Hora: 09:47:52, Número: 12
Cor: green, Data: 25/08/2023, Hora: 09:48:18, Número: 13
Cor: red, Data: 25/08/2023, Hora: 09:48:45, Número: 7
Cor: red, Data: 25/08/2023, Hora: 09:49:12, Número: 5
Cor: red, Data: 25/08/2023, Hora: 09:49:38, Número: 2
Cor: green, Data: 25/08/2023, Hora: 09:50:05, Número: 10
Cor: green, Data: 25/08/2023, Hora: 09:50:31, Número: 8
Cor: green, Data: 25/08/2023, Hora: 09:50:58, Número: 12
Cor: green, Data: 25/08/2023, Hora: 09:51:24, Número: 9
Cor: red, Data: 25/08/2023, Hora: 09:51:51, Número: 7
Cor: red, Data: 25/08/2023, Hora: 09:52:17, Número: 

In [ ]:

# Importing the necessary libraries for data manipulation and machine learning
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import queue

class Calc:
    def __init__(self):
        self.data = []
        self.model = RandomForestClassifier(n_estimators=50)
        self.is_trained = False

    def add_data(self, line):
        """
        Adds a new data line to the data list.
        """
        cor, data, hora, numero = [item.split(":")[1].strip() for item in line.split(",")]
        self.data.append({
            'Cor': cor,
            'Data': data,
            'Hora': hora,
            'Número': int(numero)
        })
        # Limit the data to the last 20 records
        self.data = self.data[-20:]

    def prepare_data(self):
        """
        Prepares the data for training the machine learning model.
        """
        if len(self.data) < 20:
            return None, None
        df = pd.DataFrame(self.data)
        # Convert categorical feature 'Cor' to numerical
        df['Cor'] = df['Cor'].astype('category').cat.codes
        # Extract hour and minute from 'Hora'
        df['Hora'] = df['Hora'].apply(lambda x: int(x.split(":")[0]) * 60 + int(x.split(":")[1]))
        X = df.drop('Cor', axis=1)
        y = df['Cor']
        return X, y

    def train_model(self):
        """
        Trains the machine learning model.
        """
        X, y = self.prepare_data()
        if X is None:
            return "Not enough data to train the model."
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.model.fit(X_train, y_train)
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        self.is_trained = True
        return f"Model trained with an accuracy of {accuracy * 100:.2f}%."

    def predict_color(self):
        """
        Predicts the next color using the trained model.
        """
        if not self.is_trained:
            return "Model is not trained yet."
        X, _ = self.prepare_data()
        if X is None:
            return "Not enough data for prediction."
        last_X = X.iloc[-1].values.reshape(1, -1)
        prediction = self.model.predict(last_X)
        # Convert numerical prediction back to color
        color_map = {0: 'green', 1: 'red'}
        return color_map[prediction[0]]
